# Extracción de datos

Primero procederemos a extraer los datos de los clientes del año pasado.

In [0]:
%sql
-- Usando el esquema tfm_staging
USE tfm_staging

In [0]:
%sql
-- Eliminamos las vistas temporales
DROP VIEW IF EXISTS temp_view_sales_client_52_weeks;
DROP VIEW IF EXISTS temp_view_client_new_category;

In [0]:
%sql
SELECT COUNT(DISTINCT CLIENTE_ID) FROM tfm_staging.stg_internal_sales WHERE CALENDAR_DAY>='2021-06-21' AND CALENDAR_DAY<='2022-06-19'  AND INGRESOBRUTO>0 AND  RM1 IN ("OP Tradicional","HM Tradicional");

count(DISTINCT CLIENTE_ID)
23985


In [0]:
%sql
-- Creamos la vista que contiene la suma de ingreso bruto del cliente por categoría del año pasado.
CREATE VIEW temp_view_sales_client_52_weeks AS
SELECT CLIENTE_ID, CATEGORIA, SUM(INGRESOBRUTO) as INGRESOBRUTO FROM tfm_staging.stg_internal_sales  
  WHERE CALENDAR_DAY>='2021-06-21' AND CALENDAR_DAY<='2022-06-19' AND INGRESOBRUTO>0 AND  RM1 IN ("OP Tradicional","HM Tradicional")
    GROUP BY CLIENTE_ID, CATEGORIA;

In [0]:
%sql
-- Creamos la vista por consumo en base al 100% de dinero del cliente por "nueva categoría"
CREATE VIEW temp_view_client_new_category AS
SELECT CLIENTE_ID,  B.NUEVA_CATEGORIA_AGRUPADA as NUEVA_CATEGORIA, ROUND(CAST((INGRESOBRUTO/SUM(INGRESOBRUTO) OVER (PARTITION BY CLIENTE_ID)) AS FLOAT),3) AS CONSUMO_CATEGORIA 
FROM temp_view_sales_client_52_weeks A LEFT JOIN tfm_staging.dim_nuevas_categorias_agrupados B on A.CATEGORIA=B.CATEGORIA
ORDER BY CLIENTE_ID